# Load libraries

In [17]:
import numpy as np
import re
import pandas as pd
import os
from datetime import datetime

# Specify git executable file for GitPython in Jupyter Notebook (In IDE, it can still work without this line.)
os.environ["GIT_PYTHON_GIT_EXECUTABLE"] = "C:\Program Files\Git\cmd\git.exe"

import git
from git import RemoteProgress

from git import Repo
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Clone repo from GitHub

Link: https://git-scm.com/docs/git-clone <br>
<b>Note:</b> In case too-long file path issue occurs in Windows, set <code>git config --system core.longpaths true</code>

In [18]:
class Progress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        print(self._cur_line)

In [19]:
remote_link = "https://github.com/apache/jackrabbit"
local_link = "C:\\Secure Software\\jackrabbit"
# Uncomment to clone
# Repo.clone_from(remote_link, local_link, progress=Progress())

In [20]:
repo = Repo(local_link)
fixing_commit = "26e601934d0f439f0a61d62265f52936d79df40d"
affected_file = "jackrabbit-webdav/src/main/java/org/apache/jackrabbit/webdav/client/methods/DavMethodBase.java"

# A) Message and Title of the Fixing Commit
Link: https://git-scm.com/docs/git-show
You can explore different parameters.

In [21]:
show_data = repo.git.show("-s", fixing_commit).splitlines()
show_data.reverse()
print("Message and title of the commit:")
print(show_data[0])

GitCommandError: Cmd('C:\Program Files\Git\cmd\git.exe') failed due to: exit code(128)
  cmdline: C:\Program Files\Git\cmd\git.exe show -s 26e601934d0f439f0a61d62265f52936d79df40d
  stderr: 'fatal: bad object 26e601934d0f439f0a61d62265f52936d79df40d'

# B) Total Files Affected

In [6]:
total_files_affected = repo.git.show("--stat",fixing_commit).splitlines()
print("Files Afftected:")
total_files_affected = total_files_affected[6:]
for line in total_files_affected:
    print(line)

GitCommandError: Cmd('C:\Program Files\Git\cmd\git.exe') failed due to: exit code(128)
  cmdline: C:\Program Files\Git\cmd\git.exe show --stat 26e601934d0f439f0a61d62265f52936d79df40d
  stderr: 'fatal: bad object 26e601934d0f439f0a61d62265f52936d79df40d'

# C) Total Directories Affected

In [7]:
total_directories_affected = repo.git.show("--dirstat",fixing_commit).splitlines()
total_directories_affected = total_directories_affected[6:]
print("Directories Affected:")
for lines in total_directories_affected:
    print(lines)

Directories Affected:
  31.1% core/src/main/java/org/apache/struts2/result/
  68.8% plugins/portlet/src/main/java/org/apache/struts2/portlet/result/


# D) Total lines of code (including comments and blank lines) Deleted

In [8]:
lines_del_inc_comm = repo.git.show("--shortstat",fixing_commit).splitlines()
lines_del_inc_comm.reverse()
lines_del_inc_comm = lines_del_inc_comm[0].split(',')
lines_del_inc_comm = lines_del_inc_comm[2].split(" ")
print("Total lines of code deleted(including comments and blank lines): "+lines_del_inc_comm[1])

Total lines of code deleted(including comments and blank lines): 3


# E) Total lines of code (including comments and blank lines) Added

In [9]:
lines_add_inc_comm = repo.git.show("--shortstat",fixing_commit).splitlines()
lines_add_inc_comm.reverse()
lines_add_inc_comm = lines_add_inc_comm[0].split(',')
lines_add_inc_comm = lines_add_inc_comm[1].split(" ")
print("Total lines of code Added(including comments and blank lines): "+lines_add_inc_comm[1])

Total lines of code Added(including comments and blank lines): 4


# F) Total lines of code (excluding comments and blank lines) Deleted

In [10]:
lines_del_exc_comm = repo.git.show("-U0","--pretty=""",fixing_commit).splitlines()
lines_added = []
tot = 0
for line in lines_del_exc_comm:
    if re.search("^\-",line):
        l = line.split(" ")
        if(len(l)>1):
            if(len(l[0]) == 1):
                if(l[1][:1] != '*' or l[1][:1] != '/'):
                    tot += 1
print("Total line of code(excluding comments and blank lines) Deleted:" + str(tot))

Total line of code(excluding comments and blank lines) Deleted:2


# G) Total lines of code (excluding comments and blank lines) Added

In [11]:
lines_add_exc_comm = repo.git.show("-U0","--pretty=""",fixing_commit).splitlines()
lines_added = []
tot = 0
for line in lines_add_exc_comm:
    if re.search("^\+",line):
        l = line.split(" ")
        if(len(l)>1):
            if(len(l[0]) == 1):
                if(l[1][:1] != '*' or l[1][:1] != '/'):
                    tot += 1
print("Total line of code(excluding comments and blank lines) Added:" + str(tot))


Total line of code(excluding comments and blank lines) Added:2


# H) Days between the current fixing commit and the previous commit

In [12]:
days = repo.git.log("--format=%cd",fixing_commit,"--",affected_file,"-2").splitlines()
list_size = len(days)
if list_size<2:
    print("0 Days")
else:
    last_commit_date = datetime.strptime(days[0],"%a %b %d %X %Y %z")
    last_prev_commit_date = datetime.strptime(days[1],"%a %b %d %X %Y %z")
    date_diff = (abs(last_commit_date-last_prev_commit_date))
    print("No of days:" + str(date_diff))

No of days:0:20:51


# I) No of times each affected file of the current fixing commit been modified in the past since their creation

In [13]:
no_of_times = repo.git.log("--follow","--format=%h",fixing_commit,"--",affected_file).splitlines()
length = len(no_of_times)
print("No of times each affected file of the current fixing commit been modified in the past since their creation:"+ str(length))

No of times each affected file of the current fixing commit been modified in the past since their creation:14


# J) Developers who has modified each affected file since its creation

In [14]:
no_of_times = repo.git.log("--format=%an",fixing_commit,"--",affected_file).splitlines()
s = set(no_of_times)
print("List of Developers:")
for line in s:
    print(line)

List of Developers:
Lukasz Lenart
Victor Sosa
victor sosa
Stefaan Dutry
Johannes Geppert


# K) Commits of each Developer

In [15]:
commit_authors = repo.git.shortlog("-sn", "--all").splitlines()
author_commits = []

for commit_author in commit_authors:
    commit, author = commit_author.split("\t")
    for authors in s:
        if(authors==author):
            author_commits.append((int(commit), authors))

In [16]:
df = pd.DataFrame(author_commits, columns=['Commit','Author'])

In [27]:
df.head()

,Commit,Author
0,2675,Filip Hanik
